<a href="https://colab.research.google.com/github/hargurjeet/DeepLearning/blob/main/GTRSB%20-%20CNN%20(TensorFlow).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# German Traffic Signal Dataset

# 1: Introduction

The German Traffic Sign Benchmark is a multi-class, single-image classification challenge held at the International Joint Conference on Neural Networks (IJCNN) 2011. Following are the details of the dataset

<br> 1. Single-image, multi-class classification problem
<br> 2. More than 40 classes
<br> 3. More than 50,000 images in total
<br> 4. Large, lifelike database

# 2: Data Pre processing

2.1 Loading the required Libraries

In [1]:
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import os
import shutil

2.4.1


In [ ]:
! pip install opendatasets --upgrade

In [5]:
from google.colab import files
files.upload()

import opendatasets as od

dataset_url = 'https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign'
od.download(dataset_url)

Saving kaggle.json to kaggle.json


  1%|          | 5.00M/612M [00:00<00:13, 46.9MB/s]

100%|██████████| 612M/612M [00:08<00:00, 79.3MB/s]


## 2.2 Get Data

In [6]:
df = pd.read_csv('/content/gtsrb-german-traffic-sign/Meta.csv')
df1 = pd.read_csv('/content/gtsrb-german-traffic-sign/Test.csv')
df2 = pd.read_csv('/content/gtsrb-german-traffic-sign/Train.csv')

In [ ]:
df.head()

In [ ]:
df1.head()

In [ ]:
df2.head()

In [7]:
## Reading the image size

import cv2

im = cv2.imread('/content/gtsrb-german-traffic-sign/Train/0/00000_00000_00000.png')

print(type(im))
# <class 'numpy.ndarray'>

print(im.shape)
print(type(im.shape))

<class 'numpy.ndarray'>
(30, 29, 3)
<class 'tuple'>


## 2.3 Data Augmentation

In [ ]:
## Setting up the Augmentation 

train_dir = '/content/gtsrb-german-traffic-sign/Train'

train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
     train_dir,  # This is the source directory for training images
     subset='training',
     target_size=(30, 30),  # All images will be resized to 150x150
     batch_size=32,
     color_mode='rgb',    
     shuffle=True,
     seed=42,
        # Since we use binary_crossentropy loss, we need binary labels
     class_mode='categorical')


test_datagen = ImageDataGenerator(rescale=1./255)

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = train_datagen.flow_from_directory(
        train_dir,
        subset='validation',
        target_size=(30, 30),
        batch_size=32,
        color_mode='rgb',    
        shuffle=True,
        seed=42,
        class_mode='categorical')

# 3. Building the CNN Model and Running the validaitons

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(30,30,3)),
    # tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'),
    # tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Dense(8, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(43, activation='softmax'),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.n//train_generator.batch_size,  
      epochs=10,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=validation_generator.n//validation_generator.batch_size)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

An accuracy of 82 percent is acheived. It is also observed from the above values of validation accuracy that the results are not over fitting.

In [ ]:
df1.head()

## 3.1 Reading the test data

In [ ]:
from shutil import copyfile


src_dir  = "/content/gtsrb-german-traffic-sign/"
test_dir = "/content/gtsrb-german-traffic-sign/Test_1/"

os.makedirs(test_dir)
for i in range(len(df1)):
    if not os.path.exists(test_dir+str(df1.iloc[i].ClassId)):
      os.makedirs(test_dir+str(df1.iloc[i].ClassId))
    copyfile(src_dir+df1.iloc[i].Path, test_dir+str(df1.iloc[i].ClassId)+'/'+df1.iloc[i].Path[5:])

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(30,30),
    batch_size=32,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

In [ ]:
STEP_SIZE_TEST =test_generator.n//test_generator.batch_size

score = model.evaluate_generator(test_generator, steps=STEP_SIZE_TEST)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
jovian.commit()